## 事前準備

### 必要なライブラリのインポート

以下のコードでは、Azure AI のプロジェクトや検索機能を操作するために必要なライブラリをインポートしています。

- dotenv： `.env` ファイルから環境変数を読み込むために使用します。
- os, json： ファイル操作や環境変数の取得に利用します。
- typing： 関数の型ヒントなどに使用します。
- azure.identity： Azure リソースへの認証を行うためのクレデンシャル（`DefaultAzureCredential`）を提供します。
- azure.core.credentials： Azure Search にアクセスする際の API キー認証に使用されます。
- azure.ai.projects： Azure AI Projects SDK を通じて、エージェントやツール、実行管理などを操作します。
- azure.search.documents： Azure AI Search のクライアントライブラリで、インデックス検索などを行います。
- azure.ai.projects.models： 検索ツール（`BingGroundingTool`）やコード実行ツール（`CodeInterpreterTool`）など、エージェントに組み込むツールを定義するためのモジュールです。


In [ ]:
import json
import os
from dotenv import load_dotenv
from typing import Any, Callable, Set
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.ai.projects import AIProjectClient
from azure.search.documents import SearchClient
from azure.ai.projects.models import FunctionTool, ToolSet, BingGroundingTool, CodeInterpreterTool
from azure.ai.projects import AIProjectClient

### 環境変数と接続設定の読み込み

このセクションでは、`.env` ファイルから各種接続情報を環境変数として読み込み、後続の処理で使用できるようにします。


In [ ]:
# 環境変数と接続設定の読み込み
load_dotenv()
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
INDEX_NAME = os.getenv("INDEX_NAME")
AI_SEARCH_KEY = os.getenv("AI_SEARCH_KEY")

PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME")

### Code Interpreter ツールの定義とクライアントの初期化

以下のコードでは、Azure AI エージェントに機能を追加するためのツール定義と、各種クライアント（AI Project、Bing、Search）を初期化しています。

主なクラス・ツールの役割

- `CodeInterpreterTool()`：エージェントが Python コードの生成・実行を行えるようにするツールです。実行結果はテキストや画像形式で取得でき、画像は保存して可視化することも可能です。

- `AIProjectClient`：Azure AI Project に接続するためのクライアントです。エージェントの管理、実行、接続情報の取得などを一括して扱うことができます。

- `DefaultAzureCredential()`：Azure リソースへのアクセスを行うための標準的な認証方式です。開発中はVisual Studio CodeやAzure CLIの認証情報を使い、本番環境ではマネージドIDを自動的に利用します。

- `project_client.connections.get()`：AI Project に登録されている接続情報（例：Bing 接続）を取得するためのメソッドです。エージェントで外部APIを利用する際に必要になります。

- `BingGroundingTool`：エージェントが Bing を使ってインターネット検索を実行できるようにするツールです。最新のWeb情報に基づいた回答が可能になります。

- `SearchClient`：Azure Cognitive Search にアクセスするためのクライアントです。インデックスに基づいてデータ検索を行うため、RAG（検索拡張生成）構成にも活用できます。

In [ ]:
# Code Interpreter ツールの定義
codeinterpreter = CodeInterpreterTool()

# Azure AI Project と Search クライアントの初期化
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=BING_CONNECTION_NAME
)  # ここに接続名を指定
bing_conn_id = bing_connection.id
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=bing_conn_id)

search_client = SearchClient(
    endpoint=AI_SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(AI_SEARCH_KEY),
)

### Run Step のタイプと出力内容の解説
この関数 `print_run_steps` は、エージェント実行（Run）の各ステップを詳細に出力するユーティリティです。Azure AI Agent Service において、Run の中で何が実行されたかを確認・デバッグするのに非常に有効です。

- `tool_calls`：エージェントが呼び出したツールに関する情報を表示します。  
  以下のツールタイプが含まれます：
  - 関数（Function Tool）
  - Bing 検索（Bing Grounding Tool）
  - コード実行（Code Interpreter Tool）

- `message_creation`：エージェントが生成したメッセージ（テキストまたは画像）を表示します。ユーザーへの応答内容が確認できます。

Code Interpreter の出力

- 実行された Python コードの内容が表示されます。
- 出力が画像の場合は、ファイルとしてローカルに保存されます。
- 出力がテキストの場合は、コンソールに直接表示されます。

Bing Grounding の出力

- エージェントが実行した Bing 検索のリクエスト URL を表示します。
- どのようなクエリが実行されたかを確認するのに役立ちます。


In [ ]:
from pprint import pprint

def print_run_steps(run_id: str, thread_id: str):
    """
    Run Step の詳細を表示する関数
    :param run_id: Run ID
    :rtype run_id: str
    :param thread: Thread ID
    :rtype run_id: str
    """
    ren_step_list = project_client.agents.list_run_steps(thread_id=thread_id, run_id=run_id)
    run_steps = reversed(ren_step_list["data"])

    for idx, step in enumerate(run_steps, start=1):
        print(f"\n=== Run Step {idx} ===")
        print(f"Type: {step['type']}")
        step_details = step["step_details"]
        step_type = step_details.get("type")
        print("Details:")
        if step_type == "tool_calls":
            for call in step_details.get("tool_calls", []):
                call_type = call.get("type")
                if call_type == "function":
                    function = call.get("function", {})
                    print(f"- Function Name: {function.get('name')}")
                    print(f"- Arguments: {function.get('arguments')}")
                elif call_type == "bing_grounding":
                    bing = call.get("bing_grounding", {})
                    print(f"- Bing Request URL: {bing.get('requesturl')}")
                elif call_type == "code_interpreter":
                    code = call.get("code_interpreter", {})
                    print(f"---- Generated code ---- :\n {code.get('input')}")
                    outputs = code.get("outputs", [])
                    for output in outputs:
                        if output.get("type") == "image":
                            file_id = output.get("image", {}).get("file_id")
                            print(f"- Generated file (image): {file_id}")
                            project_client.agents.save_file(file_id=file_id, target_dir="../sampledata", file_name="code-interpreter.png")
                            break
                        elif output.get("type") == "text":
                            print(f"- Output text: {output.get('text')}")
        elif step_type == "message_creation":
            message_id = step_details.get("message_creation").get("message_id")
            message = project_client.agents.get_message(thread_id=thread_id, message_id=message_id)
            if message.content[0].type == "text":
                print(message.content[0].text.value)
            elif message.content[0].type == "image":
                print("image generated") 
        else:
            pprint(step_details, indent=4, width=120)


## Azure AI Search を検索するカスタム関数の定義

### テキスト整形関数の定義

改行や特定の文字を置換して、整形済みのテキストを返すシンプルなユーティリティ関数です。

**テキスト整形の処理内容**

- 改行コードの除去：  `\n` や `\r` をスペース（`" "`）に置き換え、1行の読みやすいテキストに整形します。

- 角カッコの変換  ：  `"["` → `"【"`、 `"]"` → `"】"` に置き換え、日本語に適した強調表現にします。

In [ ]:
# テキスト整形関数の定義
def nonewlines(s: str) -> str:
    return s.replace("\n", " ").replace("\r", " ").replace("[", "【").replace("]", "】")

### 保険商品検索関数 `product_search` の定義

この関数は、ユーザーのクエリに基づいて Azure AI Search を使用し、保険商品の情報を検索・取得するためのものです。

**処理の流れ**

- `search_client.search()`： Azure Cognitive Search に対して、セマンティック検索でクエリを実行します。
 

- `nonewlines(doc["chunk"])`：各検索結果のテキスト（chunk）から改行や特殊文字を整形します。複数行のテキストを 1 行にし、日本語の表記ルールに合わせて角カッコも変換します。

- `json.dumps({"context": context})`：整形済みの検索結果を JSON 形式にまとめて返却します。

**`semantic_configuration_name` について**

- `semantic_configuration_name="insurance-product-info-semantic-configuration"` は、Azure Search インデックスに事前に定義されたセマンティック構成の名前です。
- 利用している Azure Cognitive Search インスタンスで定義されていない場合、エラーになります。
- 必ず 自身の環境に合わせた構成名に置き換えてください。


In [ ]:
# semantic_configuration_name は適宜いれてください
def product_search(query: str) -> str:
    """
    保険の商品に関する質問に関して、Azure AI Search の検索結果を返します。
    :param query (str): 保険の商品を検索する際のクエリ
    :rtype: str

    :return: JSON 文字列の検索結果の情報
    :rtype: str
    """

    results = search_client.search(
        search_text=query,
        query_type="semantic", 
        semantic_configuration_name=f"{INDEX_NAME}-semantic-configuration",  # セマンティック検索用の構成名（環境に合わせて変更
        top=3, 
    )

    context = [nonewlines(doc["chunk"]) for doc in results]
    context_json = json.dumps({"context": context})
    # print(context_json)
    return context_json

## エージェントの作成
Azure AI Agent Service を用いて、保険商品検索に対応する AI エージェントを作成します。
Bing 検索や Code Interpreter と連携し、マルチツールな応答が可能な構成です。

**Toolset の作成と関数の登録**

- `FunctionTool`  ：`product_search` 関数を AI に組み込むためのラッパーです。関数をエージェントで呼び出せるようにします。

- `ToolSet`  ：複数のツール（FunctionTool、CodeInterpreterTool、BingGroundingTool など）をまとめて管理するコンテナです。エージェントに渡す機能のセットを構成します。

- `.add(...)`  ：`ToolSet` に個別のツールを追加するメソッドです。検索関数、コード実行、Bing検索といったツールを登録します。


**エージェント作成時のパラメータ解説**

- `model="gpt-4o"` ：使用するモデルは GPT-4o（高速・マルチモーダル対応）。高性能な応答を実現できます。

- `name` ：このエージェントの識別名を指定します（例："Product Search Agent"）。用途に応じたわかりやすい名前を推奨します。

- `instructions` ：エージェントが従うべき行動方針を記述します。以下の点を含めることができます：
  - 保険商品の検索、Bing検索、コード実行の役割を明記
  - 日本語での対話に対応することを指定
  - 不要な改行や情報追加を禁止して、応答の一貫性を保つルールを設定

- `toolset=toolset` ：作成した `ToolSet` をこのエージェントに割り当てます。これにより、エージェントは複数の機能を利用できるようになります。

- `headers={"x-ms-enable-preview": "true"}` ：プレビュー機能（例：特定モデルや機能）を有効化するための設定です。必要に応じて追加します。



In [ ]:
# Toolset の作成＆関数の追加
user_functions: Set[Callable[..., Any]] = {product_search}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
toolset.add(codeinterpreter)
toolset.add(bing)

# Agent の作成
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Product Search Agent",
    instructions="""
あなたは、丁寧なアシスタントです。あなたは以下の業務を遂行します。
- 保険の商品について検索して回答します
- Bing 検索でインターネットの情報を検索して回答します
- Code Interpreter でコードの生成および実行を行います

# 回答時のルール
- 関数を呼び出した場合、回答の最後に改行をいれたうえで Called Function : "関数名" と表記してください

# 制約事項
- ユーザーからのメッセージは日本語で入力されます
- ユーザーからのメッセージから忠実に情報を抽出し、それに基づいて応答を生成します。
- ユーザーからのメッセージに勝手に情報を追加したり、不要な改行文字 \n を追加してはいけません

""",
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
)
print(f"agent を新規作成しました。AGENT_ID: {agent.id}")

※なお、関数内に doc string 形式で入力した内容は agent の定義として discription パラメータに自動で抽出されます。自然言語で指定できる点は便利ですし、ツールの説明を簡潔にわかりやすく表現するプロンプトエンジニアリングが重要となります。

In [ ]:
[
    print(json.dumps(tool.as_dict(), indent=4, ensure_ascii=False))
    for tool in agent.tools
    if tool.type == "function"
]

### 確認用：CodeInterpreter & Bing Search の動作テスト


In [ ]:
# スレッド作成
thread = project_client.agents.create_thread()

In [ ]:
# Code Interpreter テスト
# ファイルが生成された場合、sampledata/code-interpreter.png として保存されます。code-interpreter-sample.png はサンプルの出力結果です。
# 今回の保険業界シナリオとは全く関係ないプロンプトですが、ファイルアップロードと組みあわせて実行することも可能です。
code_msg = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="関数 f(x) = (x**3 * exp(-x**2)) / (1 + x**2) を定義し、描画してください",  
)
code_run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)

print_run_steps(run_id=code_run.id, thread_id=thread.id)

In [ ]:
# Bing Search テスト
bing_msg = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="2025年の生命保険業界のトレンドをネットで検索して教えてください。", 
)
bing_run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)

print_run_steps(run_id=bing_run.id, thread_id=thread.id)

### スレッドの作成・実行・削除
Azure AI Agent Service を用いた実行の一連の流れを示します。

**スレッド作成とメッセージ送信**

- `create_message()`：指定したスレッドに対して、ユーザーの入力メッセージ（プロンプト）を送信します。

- `role="user"`：ユーザーからの入力であることを明示します。  これにより、エージェントが人間の発言として処理します。

- `content` ：実際の問い合わせ内容を指定します。 例：`"安心保障プランの詳細を教えて"` のような自然な言語による質問。

**クリーンアップ関数**

- `delete_thread()`：スレッド（＝エージェントとの対話履歴）を削除します。複数回のやりとりが記録されたスレッド全体をクリーンに削除したいときに使用します。

- `delete_agent()`：作成したエージェント自体を削除します。不要なリソースを Azure 上に残さないために、実験・検証後にはこの処理を行うことが推奨されます。

In [ ]:
# スレッド作成とメッセージ送信
# user_message = input("タスクを入力してください：")
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="安心保障プランの詳細を教えて",
)

# メッセージの送信
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)
print_run_steps(run_id=run.id, thread_id=thread.id)

In [ ]:
# スレッドとエージェントの削除
project_client.agents.delete_thread(thread_id=thread.id)
project_client.agents.delete_agent(agent_id=agent.id)
print("Agent と Thread を削除しました。")